#### Delta Lake does not allow changing partition columns on an existing table. To introduce partitioning, I recreate the table with the new partition layout

In [0]:
%fs
ls /Volumes/workspace/ecommerce/ecommerce_data

path,name,size,modificationTime
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv,2019-Nov.csv,9006762395,1767962633000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv,2019-Oct.csv,5668612855,1767962553000
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/delta/,delta/,0,1768718421158
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/incremental/,incremental/,0,1768718421158
dbfs:/Volumes/workspace/ecommerce/ecommerce_data/parquet/,parquet/,0,1768718421158


In [0]:
df_oct = spark.read.parquet("/Volumes/workspace/ecommerce/ecommerce_data/parquet/oct/")
df_oct.limit(5).show()

+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-13 06:25:46|      view|   1002544|2053013555631882655|electronics.smart...|   apple| 460.51|518958788|e7e27c5c-1e78-481...|
|2019-10-13 06:25:46|      view|   3700301|2053013565983425517|appliances.enviro...|   vitek| 120.93|557977070|7afc206c-7259-4be...|
|2019-10-13 06:25:46|      view|  49100004|2127425375913902544|                NULL|    NULL|  45.05|514456508|9d6837a5-40df-49d...|
|2019-10-13 06:25:46|      view|   9200409|2053013552913973497|computers.periphe...|defender|  12.56|512530774|df2d048d-c1ae-41b...|
|2019-10-13 06:25:46|      view|   1306558|2053013558920217191|  comp

In [0]:
from pyspark.sql.functions import to_date, col

silver_df = spark.table("ecommerce.silver.events")

# Add derived date column
silver_df = silver_df.withColumn("event_date",to_date(col("event_time")))

# Overwrite with schema evolution enabled
silver_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("ecommerce.silver.events")

## mergeSchema does NOT allow partition changes
## Partition layout is immutable

In [0]:
silver_df.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .partitionBy("event_date") \
    .saveAsTable("ecommerce.silver.events")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7033927646047452>, line 5
      1 silver_df.write.format("delta") \
      2     .mode("overwrite") \
      3     .option("mergeSchema", "true") \
      4     .partitionBy("event_date") \
----> 5     .saveAsTable("ecommerce.silver.events")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.command(self._spark.client), self._write.observations
    739 )
    740 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observati

In [0]:
%sql
DROP TABLE ecommerce.silver.events;

In [0]:
from pyspark.sql.functions import to_date, col

silver_df = spark.table("workspace.ecommerce.silver_events")

silver_df = silver_df.withColumn(
    "event_date",
    to_date(col("event_time"))
)

silver_df.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("event_date") \
    .saveAsTable("ecommerce.silver.events")

In [0]:
%sql
OPTIMIZE ecommerce.silver.events
ZORDER BY (event_type, category_code, user_id);

path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 31, List(minCubeSize(107374182400), List(0, 0), List(31, 1374201222), 0, List(0, 0), 0, null), null, 0, 0, 31, 31, false, 0, 0, 1768718285703, 1768718287425, 8, 0, null, List(0, 0), null, 10, 10, 0, 0, null)"
